In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import random
import time
import sys

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def generate_cities(num_cities):
    cities = np.random.rand(num_cities, 2)
    return cities

In [ ]:
def distance(city1, city2):
    return np.sqrt(np.sum((city1 - city2)**2))

In [ ]:
def calculate_fitness(chromosome, cities):
    fitness = 0
    for i in range(len(chromosome) - 1):
        fitness += distance(cities[chromosome[i]], cities[chromosome[i+1]])
    fitness += distance(cities[chromosome[-1]], cities[chromosome[0]])
    return fitness

In [ ]:
def generate_population(pop_size, num_cities):
    population = []
    for _ in range(pop_size):
        chromosome = list(np.random.permutation(num_cities))
        population.append(chromosome)
    return population

In [ ]:
def selection(population, fitnesses, tournament_size=3):
    selected = random.sample(list(zip(population, fitnesses)), tournament_size)
    selected.sort(key=lambda x: x[1])
    return selected[0][0]

In [ ]:
def cyclic_crossover(parent1, parent2):
    size = len(parent1)
    child = [None] * size
    cycle_idx = 0
    start = 0

    while None in child:
        current = start
        cycle_value = parent1[start]

        while child[current] is None:
            child[current] = parent1[current]
            current = parent1.index(parent2[current])

        for i in range(size):
            if child[i] is None:
                start = i
                break
        cycle_idx += 1

    return child

In [ ]:
def ordinal_crossover(parent1, parent2):
    size = len(parent1)
    child = [None] * size
    index_map = [None] * size

    for i in range(size):
        index_map[parent1[i]] = i

    for i in range(size):
        pos = index_map[parent2[i]]
        while child[pos] is not None:
            pos = (pos + 1) % size
        child[pos] = parent2[i]

    return child

In [ ]:
def mutate(chromosome, mutation_rate=0.1):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(chromosome)), 2)
        chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
    return chromosome

In [ ]:
def genetic_algorithm(num_cities, crossover_type='cyclic', pop_size=100, generations=1000, mutation_rate=0.1):
    cities = generate_cities(num_cities)
    population = generate_population(pop_size, num_cities)
    best_fitness = float('inf')
    best_solution = None

    for generation in range(generations):
        fitnesses = [calculate_fitness(chromosome, cities) for chromosome in population]
        new_population = []

        for _ in range(pop_size // 2):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)

            if crossover_type == 'cyclic':
                child1 = cyclic_crossover(parent1, parent2)
                child2 = cyclic_crossover(parent2, parent1)
            elif crossover_type == 'ordinal':
                child1 = ordinal_crossover(parent1, parent2)
                child2 = ordinal_crossover(parent2, parent1)

            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.append(child1)
            new_population.append(child2)

        population = new_population

        generation_best_fitness = min(fitnesses)
        if generation_best_fitness < best_fitness:
            best_fitness = generation_best_fitness
            best_solution = population[fitnesses.index(best_fitness)]

    return best_solution, best_fitness

In [ ]:
def compare_crossover_time(num_cities=10, generations=1000):
    print(f"Running comparison for {num_cities} cities...")

    start_time = time.time()
    genetic_algorithm(num_cities=num_cities, crossover_type='cyclic', generations=generations)
    cyclic_time = time.time() - start_time
    print(f"Cyclic Crossover Time: {cyclic_time:.4f} seconds")

    start_time = time.time()
    genetic_algorithm(num_cities=num_cities, crossover_type='ordinal', generations=generations)
    ordinal_time = time.time() - start_time
    print(f"Ordinal Crossover Time: {ordinal_time:.4f} seconds")

    return cyclic_time, ordinal_time

In [ ]:
def measure_space_complexity():
    sample_population = generate_population(100, 10)
    cyclic_size = sys.getsizeof(cyclic_crossover(sample_population[0], sample_population[1]))
    ordinal_size = sys.getsizeof(ordinal_crossover(sample_population[0], sample_population[1]))

    print(f"Memory used by Cyclic Crossover: {cyclic_size} bytes")
    print(f"Memory used by Ordinal Crossover: {ordinal_size} bytes")

    return cyclic_size, ordinal_size

In [ ]:
if __name__ == "__main__":
    compare_crossover_time(num_cities=10, generations=1000)
    measure_space_complexity()

Running comparison for 10 cities...
Cyclic Crossover Time: 11.0496 seconds
Ordinal Crossover Time: 9.6903 seconds
Memory used by Cyclic Crossover: 136 bytes
Memory used by Ordinal Crossover: 136 bytes


In [ ]:
def run_experiments():
    city_sizes = [5, 10, 20, 40]
    for city_size in city_sizes:
        print(f"\nRunning GA for {city_size} cities...")
        best_solution, best_fitness = genetic_algorithm(num_cities=city_size)
        print(f"Best path for {city_size} cities has fitness {best_fitness:.4f}")

In [ ]:
if __name__ == "__main__":
    run_experiments()


Running GA for 5 cities...
Best path for 5 cities has fitness 1.9902

Running GA for 10 cities...
Best path for 10 cities has fitness 2.8387

Running GA for 20 cities...
Best path for 20 cities has fitness 5.1589

Running GA for 40 cities...
Best path for 40 cities has fitness 7.2633


##**CONCLUSION**

The genetic algorithm successfully finds near-optimal solutions for the Traveling Salesperson Problem (TSP) by iteratively evolving a population of potential solutions.

The algorithm utilizes key components like generating a random population, calculating fitness, selecting parents, applying crossover operators (cyclic and ordinal), and mutating offspring.

Cyclic crossover and ordinal crossover were compared in terms of time complexity and space complexity.

The results show that cyclic crossover generally performs better in terms of time complexity, while ordinal crossover exhibits slightly lower space complexity.

The choice between the two crossover operators depends on the specific constraints of the problem and the available resources.

Increasing the number of cities generally increases the complexity of the problem, requiring more generations and potentially a larger population size for convergence.

This implementation provides a basic framework for solving the TSP using a genetic algorithm. Further enhancements could include exploring different crossover operators, mutation strategies, selection methods, and population size adjustments to optimize performance for specific problem instances.